In [1]:
import numpy as np
import pandas as pd

# Define functions

In [2]:
def frequency(m, y, c):
    count = 0
    for i in range(m):
        if y[i] == c:
            count +=1
    return count

In [3]:
def probability_with_smoothing(m, num_labels, y, c):
    frequency_c = frequency(m, y, c)
    # print('Frequency:', frequency_c)
    probability_class = (frequency_c + 1) / (m + num_labels)
    # print('Probability', i+1, ':', probability_class)
    return frequency_c, probability_class

In [4]:
def frequency_class(x, column_X, y, c):
    m = column_X.shape[0]
    count = 0
    for i in range(m):
        if (column_X[i] == x) and (y[i] == c):
            count +=1
    # print('Count:', count)
    return count

In [5]:
def conditional_probability_with_smoothing(column_X, y, c, num_labels, frequency_c):
    sum_prob = 0
    column_X_set = np.unique(column_X)
    for i in range(len(column_X_set)):
        x = column_X_set[i]
        frequency_x_class = frequency_class(x, column_X, y, c)
        # print('Frequency:', frequency_x_class)
        probability_x_class = (frequency_x_class + 1) / (frequency_c + num_labels)
        # print('probability_x_class', i+1, ':', probability_x_class)
        # print('log', i+1, ':', np.log(probability_x_class))
        sum_prob += np.log(probability_x_class)
    return sum_prob

In [6]:
def compute_prediction(X, y, c, frequency_c, num_labels, probability_class):
    [m, col] = X.shape  
    sum_prob = 0
    for i in range(col):            
        sum_prob += conditional_probability_with_smoothing(X[:, i], y, c, num_labels, frequency_c)
    prediction = probability_class * sum_prob
    # print('Compute prediction:', prediction)
    return prediction

# Import data from text file and apply naive bayes algorithm

In [7]:
csvdata = np.matrix(pd.read_csv("iris.csv", header=None))
num_labels = 3
c = csvdata.shape[1]
X = np.array(csvdata[:, 0:(c - 1)])
y = np.array(csvdata[:, (c - 1)])

set_size = y.size
rand_indices = np.random.permutation(set_size)

X = X[rand_indices, :]
y = y[rand_indices]

train_size = np.ceil(set_size * 0.7).astype('uint8')

X_train = X[0:train_size, :].copy()
y_train = y[0:train_size].copy()

m = X_train.shape[0]
print('Number of examples:', m)

y_set = np.unique(y_train)
print('y_set:', y_set)
predictions = []
for i in range(len(y_set)):
    [frequency_c, probability_class] = probability_with_smoothing(m, num_labels, y_train, y_set[i])
    print('frequency_c', i+1, ':', frequency_c)
    predictions.append(compute_prediction(X_train, y_train, y_set[i], frequency_c, num_labels, probability_class))
denominator = np.sum(predictions)

max_prob = 0
max_i = 0
for i in range(len(predictions)):
    probability = predictions[i] / denominator
    if probability > max_prob:
        max_prob = probability
        max_i = i
    print('Prediction', i+1, ':', predictions[i])
    print('Probability', i+1, ':', probability)
print('Flower with max prob:', max_i+1)

Number of examples: 105
y_set: [1. 2. 3.]
frequency_c 1 : 35
frequency_c 2 : 38
frequency_c 3 : 32
Prediction 1 : -118.34639638640783
Probability 1 : 0.3411913926278408
Prediction 2 : -125.45075852907742
Probability 2 : 0.36167319255756253
Prediction 3 : -103.06504308695908
Probability 3 : 0.29713541481459677
Flower with max prob: 2
